## Wrangling Code  - The following code is constrcuted on the combined data from the Julia output

## Summary of what is inside :

#### ->The Duplicate entries removed 

#### ->removal of unwanted data inside using pattern expressions

#### ->Distance claculation

#### ->Filter out only first 3 distance

In [ ]:
#library
library(tibble)
library(dplyr)
library(tidyverse)
library(jsonlite)
library(stringr)
library(lubridate)
library(DataCombine)
library(SDMTools)
library(rgeos)
library(maptools)

### The data loaded is the output from Julia 

In [ ]:
#Load Data
Schools <- read.csv("school_data.csv")
Hospital <- read.csv("hospital_data.csv")

#remove Duplicates if at all missed in the extrcation
Hospital <- Hospital[!duplicated(Hospital$name), ]

Schools <- Schools[!duplicated(Schools$name), ]


#### The Duplicates are removed just so to deleted any redendunt values that where missed during extraction

In [ ]:
#patterns of word removal
#School - pattern
ptn = '^Eye.*?'
#Train
#center
##care
#Institute
#Dental
#Studio -147
#tech
#training
#Health
#Life Coach
#, Art Soc. rooms.
# Commercial Pilot Academy
#NZ Modern School of Music - Hawkes Bay
#Swim
#Taradale dirt park & pump track
#club
##Centre
#company
# Dance
#Aquatic
#Art Works
#Tuition
#ETC - English Language School in New Zealand
#School of
#Review
#Arts
ndx = grep("Centre+", Schools$name, perl = T)
selected_rows = Schools[ndx, ]
Schools <- Schools[-ndx, ]

#Hospital


#Eye
#Home
#train
#Cancer
#oncology
#Rehab
#Respiratory
#Endosopy
#Dental
#Service
#Distribution
#carpark
#Cycle
#spa
#Wellness
#Retirement
# Store
#Station
#board
#Planning
#Cycle
#Village
#Christchurch Pain Managment
#Forensic
#Christchurch Methadone Program
#Drug
ndx = grep("Drug+", Hospital$name, perl = T)
selected_rows = Hospital[ndx, ]
Hospital <- Hospital[-ndx, ]
Hospital <- Hospital %>% select(-X)


Each the dataframes has few "unwanted" data in them hence grep was used to remove any and all of the pattens matched

In [ ]:
#check schools list :

gov_data<-read.csv("gov_data_schools.csv")
common_Schools<-merge(x = Schools,y=gov_data,by.x ="school_name",by.y="Org.Name")


#### This was used only for the purpose of checking how many of the schools extracted are in govenrment data.
#### Due to the reason there are no common filed to join except for name which might not be unique and  the data is not joined.


In [ ]:

#write the data to folder
write.csv(Hospital, "clean_hospital.csv", row.names = FALSE)
write.csv(Schools, "clean_schools.csv", row.names = FALSE)


In [ ]:
######DISTANCE ############

Hospital$name<-as.character(Hospital$hospital_name)
Schools$name<-as.character(Schools$school_name)
Hospital$city<-as.character(Hospital$city)
Schools$city<-as.character(Schools$city)


In [2]:
#the below function calculated the distance between 2 given latitude and longitude and stores it in to new dataframe

shortes_3 <- data.frame()
sorted_distance <- data.frame()
shortes_dis <- data.frame()
distance_cacl<-function(Schools,Hospitals){
    for (i in 1:nrow(Schools)) {
      city_val <- Schools[i, ]$city
      print(Schools[i, ]$name)
      city_vise <- Hospital %>% filter(city == city_val)
        for (j in 1:nrow(city_vise)) {
            Vincenty <-
          distance(
            lat1 = Schools[i, ]$lat,
            lon1 = Schools[i, ]$long,
            lat2 = city_vise[j, ]$lat,
            lon2 = city_vise[j, ]$long
              )
        DirectDistance <- as.numeric(Vincenty[5])
        distance <- DirectDistance / 1000
        if (distance < 5) {
          df <-
            data.frame(
              school = as.character(Schools[i, ]$name),
              distance = distance,
              hospital = as.character(Hospital[j, ]$name),
              school_lat = Schools[i, ]$lat,
              school_long = Schools[i, ]$long,
              city = city_val
            )
          if (nrow(shortes_dis) == 0) {
            shortes_dis <- df
            }
          else{
            shortes_dis <- InsertRow(shortes_dis, NewRow = df)}
    }
        else{
          print(Schools[i, ]$name)} }
  
      sorted_distance <- shortes_dis[order(shortes_dis$distance), ]
      #the following snippet of code picks only the first 3 sorted entries of data and stores it into a 
        #dataframe as the loop progresses 
        if (nrow(shortes_3) == 0) {
        shortes_3 <- (sorted_distance[1:3, ])
      }
      else{
        shortes_3 <- InsertRow(shortes_3, sorted_distance[1:3, ])
      }
      shortes_dis <- shortes_dis[0, ]
      sorted_distance <- sorted_distance[0, ]}
}


In [ ]:
#create a row number for easy iteration
distance_df<- shortes_3 %>% mutate(id = row_number())

In [ ]:
#remove the unnecessary null values got while calcualting distance.
count(is.na(distance_df$school))

clean_distance_df<-distance_df[complete.cases(distance_df$school),]


In [ ]:
write.csv(clean_distance_df,"clean_shortest_distance.csv",row.names =  FALSE)